<a href="https://colab.research.google.com/github/Daniel-Brett-Stark/PortfolioProjects/blob/main/MachineLearningModelEvaluationProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%matplotlib inline

In [2]:
import numpy as np

## 1. Question to answer

Would two people with identical observed characteristics but different race be predicted to have their loan applications treated differently?

## 2. Obtain a labeled dataset

In [3]:
import pandas as pd


In [4]:
#Don't do restart runtime. Just rerun it. You need to upload mortgagedenials.csv every time I use this
from google.colab import drive
drive.mount('/content/gdrive')
df1 = pd.read_csv('/content/gdrive/My Drive/Econ 484/mortgagedenials.csv')
#MortgageDenials=pd.read_csv('mortgagedenials.csv')
MortageDenials = df1
y = MortageDenials.filter(items=['deny'])
#filter-->These are the columns you want to use for y (specified)
X = MortageDenials.drop(columns='deny')
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
print('our y vector is:\n',y.head)
print('our X matrix is:\n',X.head)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

Mounted at /content/gdrive
our y vector is:
 <bound method NDFrame.head of       deny
0        0
1        0
2        1
3        0
4        0
...    ...
2375     0
2376     1
2377     0
2378     0
2379     0

[2380 rows x 1 columns]>
our X matrix is:
 <bound method NDFrame.head of       p_irat  black  hse_inc  loan_val  ccred  mcred  pubrec  denpmi  selfemp  \
0     0.2170      0   0.2130  0.638889      1      1       0       0        0   
1     0.2600      0   0.2200  0.945946      6      2       0       0        0   
2     0.4600      0   0.2700  0.840000      5      2       0       0        0   
3     0.4900      0   0.2700  0.199482      1      1       0       0        0   
4     0.2600      0   0.2600  0.366667      1      2       0       0        0   
...      ...    ...      ...       ...    ...    ...     ...     ...      ...   
2375  0.2800      1   0.2400  0.438462      1      2       0       0        0   
2376  0.3207      1   0.2522  0.900000      1      2       0       0   

In [5]:
y = MortageDenials.filter(items=['deny'])
#filter-->These are the columns you want to use for y (specified)
X = MortageDenials.drop(columns='deny')
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

In [6]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled.shape

(2380, 15)

## 3. Divide into training and set sets

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=.25)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=.25)


## 4. Pick an appropriate method

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
import os

## 5. Choose regularization parameters via cross-validation on the training set

In [10]:
from sklearn.model_selection import GridSearchCV  #43 min
tree=DecisionTreeClassifier()

parameters = {'max_depth':range(3,20), "min_samples_leaf":range(1,5)}
grid_search = GridSearchCV(tree, parameters, n_jobs=4)
best_model=grid_search.fit(X_train, y_train)
tree_model = grid_search.best_estimator_
print (grid_search.best_score_, grid_search.best_params_) 

optimal_tree=DecisionTreeClassifier(max_depth=5,min_samples_leaf=4)

0.8957983193277311 {'max_depth': 5, 'min_samples_leaf': 4}


In [11]:
#Do I need this? 
tree = DecisionTreeClassifier(max_depth=3, min_samples_leaf = 14,random_state=0)
tree.fit(X_train, y_train)
#Need to do cross validation in Decision Trees 

DecisionTreeClassifier(max_depth=3, min_samples_leaf=14, random_state=0)

## 6. Fit model on whole training set using the cross-validated parameters

In [12]:
optimal_tree.fit(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(optimal_tree.score(X_train, y_train)))

#https://towardsdatascience.com/how-to-find-decision-tree-depth-via-cross-validation-2bf143f0f3d6

Accuracy on training set: 0.913


## 7. Evaluate model by applying it to test set

In [13]:
y_pred=optimal_tree.predict(X_test)

In [14]:
print("Accuracy on test set: {:.3f}".format(optimal_tree.score(X_test, y_test)))

Accuracy on test set: 0.908


## 8. Repeat 4-7 for another method

In [15]:
#Step 4 Pick an appropriate method
#Choose a method appropriate for classification and import its library:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC


In [16]:
#Step 5 Part 1
#Choose regularization parameters via cross-validation on the training set
#Search over a grid of values of the regularization parameters for the parameters that perform the best on the left-out folds:

param_model = {'C': [.001,.1],'gamma': [1.5],'kernel':['poly']}
#C-->.01
#gamma-->1.3, 1.7
#kernel-->'rbf', 'linear'
grid_search = GridSearchCV(SVC(),param_model,cv=5,return_train_score=True)
best_model=grid_search.fit(X_train,y_train)
print("Best C: ",best_model.best_estimator_.get_params()['C'])
print("Best gamma: ",best_model.best_estimator_.get_params()['gamma'])
print("Best kernel: ",best_model.best_estimator_.get_params()['kernel'])

print("Accuracy on training set: {:.3f}".format(
    best_model.score(X_train, y_train)))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best C:  0.001
Best gamma:  1.5
Best kernel:  poly
Accuracy on training set: 0.917


In [18]:
#Step 5 part 2
optimal_param_model=SVC(C=.001,gamma=1.5,kernel='poly')

In [19]:
#Step 6 Fit model on whole training set using the cross-validated parameters
optimal_param_model.fit(X_train, y_train)

optimal_param_model.score(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(optimal_param_model.score(X_train, y_train)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on training set: 0.917


In [20]:
#Step 7 Evaluate model by applying it to test set
#Compute and print out the "score" of the model applied to the test set:
y_pred=optimal_param_model.predict(X_test)

print("Accuracy on test set: {:.3f}".format(optimal_param_model.score(X_test, y_test)))

Accuracy on test set: 0.908


## 9. Apply the chosen method to new observations for which we have no labels

In [21]:
y

,deny
0,0
1,0
2,1
3,0
4,0
...,...
2375,0
2376,1
2377,0
2378,0


In [22]:
#NewApplicants=pd.read_csv('newapplicants.csv')
df2 = pd.read_csv('/content/gdrive/My Drive/Econ 484/newapplicants.csv')
NewApplicants = df2
optimal_param_model.predict(NewApplicants)

#The result was the same for both applicatnts (they were both denied). Since the outcome for both was the same it suggest race does not make it more likely for someone to be denied a loan

array([1, 1])

In [23]:
NewApplicants.shape
#2 people being observed
#15 characteristics of those people being observed 

(2, 15)

In [24]:
y
#1 means deny

,deny
0,0
1,0
2,1
3,0
4,0
...,...
2375,0
2376,1
2377,0
2378,0


The spreadsheet 'newapplicants.csv' contains information on two new mortgage applicants, the first black, the second non-black, but who otherwise share identical characteristics, namely, they both have monthly debt-to-income and housing expenses to income ratio of .25, both have a loan to property value ratio of .951, both have a category 4 credit and mortgage score, neither has a public record of credit problems, neither was denied mortgage insurance, both are self-employed, both are single, neither graduated from high school, both have occupations with an industry unemployment rate of 9, and neither is purchasing a condo.

Apply the chosen model to predict whether each applicant will be denied or not, and print out the predictions. 

In [26]:

optimal_param_model.predict(NewApplicants)

array([1, 1])

Question to answer (from #1): Would two people with identical observed characteristics but different race be predicted to have their loan applications treated differently?

MY ANSWER TO THE QUESTION ABOVE: The result was the same for both applicatnts (they were both denied). Since the outcome for both was the same it suggest race does not make it more likely for someone to be denied a loan. It suggests the loan application process is fair (at least in regards to the basis of race) 